#### Archivo de preprocesamiento de datos de la boya (fuente principal)

https://www.windguru.cz/47761

Importamos librerías necesarias

In [1]:
import pandas as pd
from datetime import datetime
import os

Esta función se encarga de averiguar para que fecha se predice a partir de la fecha en la que se consulto, la hora y el dia. La devuelve en formateo datetime para calcular facilmente la antelacion posteriormente

In [2]:
def sacar_fecha_target(valor, fecha_pred):    
    # sacamos dia y hora del valor
    hora = int(valor[-3:-1])
    dia = int(valor[2:-4])

    fecha = fecha_pred
    fecha = fecha.replace(hour=hora, day=dia)
    if (dia < fecha_pred.day):
        # estan en meses o anios diferentes, ej 31 -> 1
        if (fecha_pred.month == 12): # si es diciembre -> cambiamos de anio
            fecha.replace(month=1, year=fecha_pred.year + 1)
        else:
            fecha.replace(month=fecha_pred.month + 1)

    return fecha        

Esta funcion realiza el preprocesado(renombra columnas, calcula antelacion y elimina filas pasadas)

In [3]:
def preprocess_df(df, fecha_pred):
    # transponemos la matriz
    df = df.T
    df.columns = ['Vviento', 'AlturaOlas', 'PeriodoOlas', 'Temperatura', 'Nubosidad', 'Lluvia']
    df['fecha'] = df.index.map(lambda x: sacar_fecha_target(x, fecha_pred))
    # nos quedamos con predicciones (descartamos pasado)
    df = df[df['fecha'] >= fecha_pred]
    # calculamos antelacion (en horas)
    df['antelacion'] = (pd.to_datetime(df['fecha']) - fecha_pred).dt.total_seconds() // 3600
    df['hora'] = df.fecha.dt.hour
    df['dia'] = df.fecha.dt.day
    df['anio'] = df.fecha.dt.year
    df['mes'] = df.fecha.dt.month

    df = df.drop(columns='fecha')
    df = df.reset_index(drop=True)
    # no hay na's al ser predicciones
    #df = df.fillna(0)
    #df = df[~df.map(lambda x: x == '-').any(axis=1)]
    return df

Los archivos tienen datos de predicciones. Además por alguna razón se mezclan con datos reales(sigue apareciendo una hora, cuando ya ha pasado)

Asumimos que el formato del archivo es 10h_03_05_2024.csv, donde:
- 10 - hora
- 03 - dia
- 05 - mes
- 2024 - anio

In [6]:
dirName = 'primaria-raw'
directory = os.fsencode(dirName)
dfs = []
# recorremos todos los datos sacados y los guardamos de forma preprocesada
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        # sacamos los datos de fecha en la que se guardaron las predicciones
        # para poder calcular con cuanta antelación (en horas) se hizo la predicción
        fecha = datetime.strptime(filename.replace('h', ''), '%H_%d_%m_%Y.csv')
        df = pd.read_csv(dirName + "/" + filename)
        df = preprocess_df(df, fecha)
        df['filename'] = filename
        dfs.append(df)

Creamos directorio de salida si no existe

In [7]:
outDir = 'preprocessed'
if not os.path.exists(outDir):
    os.makedirs(outDir)

Guardamos el dataframe concatenando todos los archivos en uno

In [8]:
# juntamos los df's y los guardamos
pd.concat(dfs).to_csv(outDir+'/forecast_principal.csv', index=False)

In [9]:
# revisa que se guardó bien
df = pd.read_csv(outDir+'/forecast_principal.csv')
df

,Vviento,AlturaOlas,PeriodoOlas,Temperatura,Nubosidad,Lluvia,antelacion,hora,dia,anio,mes,filename
0,7.0,0.9,4.0,10.0,100.0,1.5,0.0,14,2,2024,5,14h_02_05_2024.csv
1,7.0,0.9,4.0,10.0,100.0,1.3,1.0,15,2,2024,5,14h_02_05_2024.csv
2,6.0,0.8,4.0,10.0,100.0,0.7,2.0,16,2,2024,5,14h_02_05_2024.csv
3,5.0,0.8,4.0,10.0,100.0,0.2,3.0,17,2,2024,5,14h_02_05_2024.csv
4,5.0,0.8,5.0,10.0,100.0,,4.0,18,2,2024,5,14h_02_05_2024.csv
...,...,...,...,...,...,...,...,...,...,...,...,...
443,6.0,1.0,9.0,13.0,79.0,,40.0,12,5,2024,5,20h_03_05_2024.csv
444,7.0,1.0,9.0,14.0,72.0,,41.0,13,5,2024,5,20h_03_05_2024.csv
445,7.0,1.0,9.0,14.0,82.0,,42.0,14,5,2024,5,20h_03_05_2024.csv
446,7.0,1.0,9.0,14.0,96.0,,43.0,15,5,2024,5,20h_03_05_2024.csv
